<div align="right" vertical-align="middle" style="border: 2px solid;border-radius: 5px;background-color:lightgrey;padding:5px;padding-right:20px;padding-left:10px;">
        <a style="color:black;text-decoration:none;" title="Home" href="index.ipynb">
            <img src="../../css/iconmonstr-christmas-house-icon.svg" height = "30" width = "30" style="display:inline">
        </a>
        &nbsp;
        <b>|</b>
        &nbsp;
        <a style="color:black;text-decoration:none;" title="Build" href="build.ipynb">
            <img src="../../css/iconmonstr-puzzle-icon.svg" height = "30" width = "30" style="display:inline">
        </a>
        <a style="color:black;text-decoration:none;" title="Assemble" href="../assemble_docs/assemble.ipynb">
            <img src="../../css/iconmonstr-puzzle-17-icon.svg" height = "30" width = "30" style="display:inline">
        </a>
        <a style="color:black;text-decoration:none;" title="Query" href="../query_docs/query.ipynb">
            <img src="../../css/iconmonstr-flask-3-icon.svg" height = "30" width = "30" style="display:inline">
        </a>
</div>


<h1>TUTORIAL: Build an EGRIN 2.0 Ensemble</h1>

### In a nutshell

The `BUILD` scripts configure [cMonkey2](https://github.com/baliga-lab/cmonkey2) to run on an SGE managed cluster. In the process, they alter select cMonkey2 parameters to generate variability in each run. They also choose experiments to include in each run according to user-provided annotations.

### Requirements

User must supply several files, including:

- `ratios`: a Tab-delimited or Comma-separated file containing a matrix of gene expression values across all conditions.
- `blocks`: a Comma-separated file containing annotations for each condition.
- `inclusion_blocks`: : a Comma-separated file containing groups of blocks to be co-included in runs.
- `exclusion_blocks`: : a Comma-separated file containing groups of blocks to be co-excluded in runs.

Optionally:
- `pipeline`: a JSON file containing custom scoring pipeline. Currently only set-enrichment pipeline is supported
- `setenrich_files`: Comma-separated files containing set-enrichment sets. Multiple files should be separated by a comma.

The format for each of these files will be described in detail below.

We have provided example files for building an *Mycobacterium Tuberculosis* ensemble. These data come from the following publications:

[EJR Peterson, DJ Reiss, S Turkarslan, KJ Minch, T Rustad, CL Plaisier, WJR Longabaugh, DR Sherman, NS Baliga. (2014). A high-resolution network model for global gene regulation in Mycobacterium tuberculosis. *Nucleic Acids Research*.](http://nar.oxfordjournals.org/content/early/2014/09/17/nar.gku777.full)

[TR Rustad, KJ Minch, S Ma, JK Winkler, S Hobbes, MJ Hickey, W Brabant, S Turkarslan, ND Price, NS Baliga and DR Sherman. (2014). Mapping and manipulating the Mycobacterium tuberculosis transcriptome using a transcription factor overexpression-derived regulatory network. *Genome Biology*](http://genomebiology.com/2014/15/11/502/abstract)

[KJ Minch, TR Rustad, EJR Peterson, J Winkler, DJ Reiss, S Ma, M Hickey, W Brabant, B Morrison, S Turkaslan, C Mawhinney, JE Galagan, ND Price, NS Baliga, DR. Sherman. (2014). The DNA-binding network of Mycobacterium tuberculosis. *Nature Communications*](http://www.nature.com/ncomms/2015/150112/ncomms6829/full/ncomms6829.html)

Additionally, the Python modules described on the [Home page](../index.ipynb) are also required to run tese scripts.

### Scripts

`cMonkeyQSub.py`: The control function for `BUILD` scripts. Writes QSub script.

`cMonkeyQSub.py`: The control function for `BUILD` scripts. Writes QSub script.

`cMonkeyQSub.py`: The control function for `BUILD` scripts. Writes QSub script.



